In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import json
import pygraphviz
import numpy as np


In [137]:
class ExerciseNetwork():
    def __init__(self, initial_graph = None, json_graph = None):
        is_graph = isinstance(initial_graph, nx.DiGraph)
        is_dict = isinstance(json_graph, dict)
        if is_graph:
            self.graph = initial_graph
        elif is_dict:
            self.graph = node_link_graph(json_graph)
        else:
            self.graph = nx.DiGraph()

        
    @property
    def all_labels(self):
        return [node[1]['name'] for node in self.graph.nodes(data=True)]
    
    
    @staticmethod
    def get_common_elements(list1, list2):
        set1, set2 = set(list1), set(list2)
        return set1 & set2  # return elements in both list1 and list2

    
    @staticmethod
    def get_unique_elements(list1, list2):
        set1, set2 = set(list1), set(list2)
        unique_elements = set1 ^ set2
        return unique_elements  # return elements that do not appear in both list1 and list2
    
    
    def add_branch(self, new_labels, start_node_name):
        # check if any of the new labels are currently in the graph. No new node is added and a new edge is added.
        common_labels = self.get_common_elements(self.all_labels, new_labels)
        # Isolate new labels that are unique
        unique_labels = self.get_unique_elements(common_labels, new_labels)
        
        num = max(self.graph.nodes) + 1 # start labelling nodes from this number
        num_new_nodes = len(new_labels) - len(common_labels) # exclude new nodes from being added
        
        new_nodes = range(num, num+num_new_nodes)
        old_nodes = get_nodes_of_labels(self.graph, list(common_labels)) # find the node # of the nodes already in the graph 
        
        labels = {node_num: {'name':label} for node_num, label in zip(list(new_nodes), unique_labels)}
        # add new nodes and give them attributes from new_labels
        self.graph.add_nodes_from(new_nodes)
        nx.set_node_attributes(self.graph, labels)

        # find node to branch edges from
        node = [x for x,y in self.graph.nodes(data = True) if y['name'] == start_node_name][0]
        
        edges = [(node, new_node) for new_node in new_nodes] + [(node, old_node) for old_node in old_nodes]
        # add edges to the new nodes, and the new edges to the old nodes
        self.graph.add_edges_from(edges)
    
    
    def add_branches(self, new_labels, start_node_names):
        for label, start in zip(new_labels, start_node_names):        
            add_branch(self.graph, label, start)
       
    
    def add_branches_from_dict(self, exercise_dict):
        """
        Add branches from a dictionary with the structure:
        {start_node1:[end1, end2,...], start_node2:[end1, end2], ...}
        If an endpoint is specified twice then an edge is added to that node from all start nodes. 
        """
        for node in exercise_dict.keys():
            self.add_branch(exercise_dict[node], node)
    
    
    def get_labels_of_nodes(self, nodes):
        return [y['name'] for x,y in self.graph.nodes(data = True) if x in nodes]
    
    
    def get_nodes_of_labels(self, labels):
        return [x for x,y in self.graph.nodes(data = True) if y['name'] in labels]
     
    
    def get_children_of(self, name):
        start_node = get_nodes_of_labels(self.graph, name)[0]
        child_nodes = list(self.graph.successors(start_node))
        child_names = get_labels_of_nodes(self.graph, child_nodes)
        return child_nodes, child_names
    
    
    def get_parents_of(self, name):
        start_node = get_nodes_of_labels(self.graph, name)[0]
        parent_nodes = list(self.graph.predecessors(start_node))
        parent_names = get_labels_of_nodes(self.graph, parent_nodes)
        return parent_nodes, parent_names

    
    def add_edge_from_name(self, start_label, end_label):
        nodes = get_nodes_of_labels(self.graph, [start_label, end_label])
        edges = [(nodes[0], nodes[1])]
        self.graph.add_edges_from(edges)
    
    
    def add_edges_from_dict(self, edges_dict):
        """
        Add edges from a dictionary of nodes {from1:[dest1, dest2,...], from2:[dest1,dest2,...]}
        """
        for start_label in edges_dict.keys():
            for end_label in edges_dict[start_label]:
                add_edge_from_name(self.graph, start_label, end_label)

    
    def draw_graph(self, size = (20,20)):
        pos = graphviz_layout(self.graph, prog='twopi', args = '')
        labels = {x:y['name'] for x,y in self.graph.nodes(data = True)}
        plt.figure(figsize = size)
        nx.draw(self.graph, pos, node_size=0, alpha=0.4, edge_color="r", font_size=16, with_labels=True, labels = labels)
    
    
    def graph_to_json(self):
        return node_link_data(self.graph)
    
    
    def graph_to_file(self, filename):
        with open(filename, 'w') as outfile:
            json.dump(self.graph_to_json(), outfile)
    
    
    def file_to_graph(self, filename):
        with open(filename) as json_file:
            data = json.load(json_file)
            self.graph = node_link_graph(data)
    
    
ex1 = ExerciseNetwork(json_graph=x)

# individual exercises

exercises = {
    "Flyes": ["Floor Flyes", " Ground Pounds"],
    "Push Ups": ["Twisting Push Up", "Decline Push Up", "Close Grip Push Up"],
    "Dips": ["Weighted Dips", "Plyo Dips", "Retro Dips"],
    "Bench": ["Flat Bench Press", "Close Grip Bench Press", "Incline Bench Press"],
    "Extensions": ["Skull Crusher", "Push Aways", "Bodyweight Extensions"],
    "Shoulder Raise": ["Lateral Raise", "Front Raise", "Leaning Raise"],
    "Press": ["Standing Press", "Landmine Press", "Incline Bench Press"],
    "Pulls": ["Face Pulls", "Landmine Upper Row", "High Boy Row"],
    "Bicep Curls": ["Bicep Curl", "Incline Curl", "Waiter Curl"],
    "Rows": ["Yates Row", "Bench Row", "Dead Row"],
    "Pull Ups": ["Chin Up", "Wide Pull Up"],
    "Pull Downs": ["Straight Arm Pull Down", "Lat Pull Downs"],
    "Calf Raise": ["Forward Raise", "Straight Leg Raise", "Bent Leg Raise"],
    "Unilateral": ["Bulgarian Split Squat", "Pistol Squat", "Step Ups"],
    "Squats": ["Back Squats", "Front Squats", "Jumps"],
    "Deadlift": ["Hex Bar Deadlift", "Straight Bar Deadlift", "Romanian Dead Lift"],
    "Bridges": ["Glute Bridge"],
    "Hamstring Curls": ["Medicine Ball Curls"]
}

ex1.add_branches_from_dict(exercises)


In [1]:
%%writefile setup.py

from setuptools import setup, find_packages

setup(
    name="TrackerApp",
    version="0.1.0",
    packages=find_packages(exclude=['*test'])
)


Writing setup.py
